In [1]:
import os
import pandas as pd
from langchain_community.utilities import SQLDatabase
from gait.idris import Idris, IdrisTrainer, IdrisLiteEmb, IdrisLiteLLM, IdrisRDB
import pyspark
from typing import List
import litellm


# optional – shows the exact URL LiteLLM calls, very useful for debugging
litellm._turn_on_debug()       

# 1. Create a small test DataFrame
data = {
    'id': [1, 2, 3, 4, 5],
    'name': ['Alice', 'Bob', 'Charlie', 'David', 'Eve'],
    'age': [25, 30, 35, 40, 45],
    'department': ['HR', 'Engineering', 'Sales', 'Marketing', 'Engineering']
}
df = pd.DataFrame(data)

# 2. Set up a SQLite database for LangChain
import sqlite3
conn = sqlite3.connect('test.db')
df.to_sql('employees', conn, if_exists='replace', index=False)
db = SQLDatabase.from_uri("sqlite:///test.db")

# 3. Create trainer and generate initial pairs
trainer = IdrisTrainer()
result = trainer.train(df, "employees", limit=5)

# 4. Print initial pairs
print(f"Initial pairs count: {len(result.question_sql)}")
print("\nSample pairs:")
for nl, sql in result.question_sql[:3]:
    print(f"NL: {nl}")
    print(f"SQL: {sql}")
    print("---")

# 5. Augment with LangChain (set your OpenAI API key)
import os
from dotenv import load_dotenv
#load_dotenv()  # This will load the .env file from the project root
#os.environ["OPENAI_API_KEY"] = "4e86557a89d24a95a61d41e02cf62d1d"
env_path = r"C:\Users\jac10179\Documents\Admin\Career\2025\AI\Fork\gait-main-JE\.env"
load_dotenv(dotenv_path=env_path)

# Update your notebook test to not specify a model (use env settings)
aug_pairs = trainer.augment_pairs_with_langchain(
    result.question_sql[:3],
    db,
    model_name=None,  # This will use AZURE_OPENAI_DEPLOYMENT from env
    paraphrases_per_query=2
)

# 6. Print augmented pairs
print(f"\nAugmented pairs count: {len(aug_pairs)}")
print("\nSample augmented pairs:")
for nl, sql in aug_pairs[:3]:
    print(f"NL: {nl}")
    print(f"SQL: {sql}")
    print("---")

# 7. Try the combined training method
combined_result = trainer.train_with_langchain_augmentation(
    df, 
    "employees", 
    db,
    limit=3,
    model_name=None,  # Use env settings
    paraphrases_per_query=1
)

print(f"\nCombined training pairs count: {len(combined_result.question_sql)}")

#Create a mock RDB class
class MockRDB(IdrisRDB):
    @property
    def dialect(self) -> str:
        return "SQLite"  # or whatever dialect you want to mock
    
    def _get_create_table_columns(self, table_name: str) -> List[str]:
        # Return a mock list of columns
        return ["id INTEGER", "name VARCHAR", "age INTEGER", "department VARCHAR"]
    
    def execute_sql(self, sql: str) -> pd.DataFrame:
        print(f"Would execute: {sql}")
        # Return an empty DataFrame with the expected columns
        return pd.DataFrame(columns=["id", "name", "age", "department"])


# 8. Test with Idris instance
rdb = MockRDB()

# Use environment variables for embedding model
llm = IdrisLiteLLM(
    model_name=os.getenv("AZURE_API_DEPLOYMENT"),  # "gpt-4o"
    api_base=os.getenv("AZURE_API_BASE"),
    api_key=os.getenv("AZURE_API_KEY"),
    api_version=os.getenv("AZURE_API_VERSION")
)

# Use text-embedding-3-large for embeddings

emb = IdrisLiteEmb(
    model_name="azure/text-embedding-ada-002",
    api_base="https://ist-apim-aoai.azure-api.net/load-balancing/text-embedding-ada-002",
    api_version="2024-10-21",
    api_key="4e86557a89d24a95a61d41e02cf62d1d",
)

idris = Idris(rdb, emb, llm)
idris.add_create_table(combined_result.create_table)
idris.load_context(combined_result.context)
idris.load_question_sql(combined_result.question_sql)

# 9. Test query
sql = idris.generate_sql("Who works in Engineering?")
print(f"\nGenerated SQL: {sql}")

# 10. Test direct augmentation of an Idris instance
idris.augment_question_sql_with_langchain(
    db,
    paraphrases_per_query=1
)
print(f"\nFinal pairs count: {len(idris.dump_question_sql())}")

# After generating the pairs (either initial or augmented), add this code:

import json
from datetime import datetime

# Create a dictionary with metadata and the pairs
output_data = {
    "timestamp": datetime.now().isoformat(),
    "pairs": [
        {
            "natural_language": nl,
            "sql": sql
        }
        for nl, sql in result.question_sql  # or aug_pairs if you want to save augmented pairs
    ]
}

# Save to a JSON file with timestamp in the name
output_filename = f"sql_nl_pairs_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
with open(output_filename, 'w') as f:
    json.dump(output_data, f, indent=2)

print(f"Saved {len(output_data['pairs'])} pairs to {output_filename}")

Initial pairs count: 19

Sample pairs:
NL: Show employees where id is greater than 4
SQL: SELECT * FROM employees where id>'4'
---
NL: Show employees where id is less than 2
SQL: SELECT * FROM employees where id<'2'
---
NL: Show employees where id is less than 3
SQL: SELECT * FROM employees where id<'3'
---

Augmented pairs count: 6

Sample augmented pairs:
NL: Show employees where id is greater than 4
SQL: SELECT "id", "name", "age", "department" FROM "employees" WHERE "id" > 4 LIMIT 5;
---
NL: Show employees where id is greater than 4
SQL: SELECT "id", "name", "age", "department" FROM employees WHERE "id" > 4 LIMIT 5;
---
NL: Show employees where id is less than 2
SQL: SELECT "id", "name", "age", "department" 
FROM employees 
WHERE "id" < 2 
LIMIT 5;
---


23:12:37 - LiteLLM:DEBUG: utils.py:331 - 

23:12:37 - LiteLLM:DEBUG: utils.py:331 - Request to litellm:
23:12:37 - LiteLLM:DEBUG: utils.py:331 - litellm.embedding(input=["Use column 'id' in reference to id.", "Use column 'name' in reference to name.", "Use column 'age' in reference to age.", "Use column 'department' in reference to department."], model='azure/text-embedding-ada-002', api_base='https://ist-apim-aoai.azure-api.net/load-balancing/text-embedding-ada-002', api_version='2024-10-21', api_key='4e86557a89d24a95a61d41e02cf62d1d')
23:12:37 - LiteLLM:DEBUG: utils.py:331 - 

23:12:37 - LiteLLM:DEBUG: litellm_logging.py:427 - self.optional_params: {}
23:12:37 - LiteLLM:DEBUG: utils.py:331 - SYNC kwargs[caching]: False; litellm.cache: None; kwargs.get('cache')['no-cache']: False
23:12:37 - LiteLLM:DEBUG: litellm_logging.py:427 - self.optional_params: {}
23:12:37 - LiteLLM:DEBUG: litellm_logging.py:870 - {'litellm_trace_id': None, 'litellm_call_id': 'd3508da4-d6dd-4724-99a2-ffeb2388d0


Combined training pairs count: 24


23:12:37 - LiteLLM:DEBUG: common_utils.py:384 - Initializing Azure OpenAI Client for text-embedding-ada-002, Api Base: https://ist-apim-aoai.azure-api.net/load-balancing/text-embedding-ada-002, Api Key:4e86557a***************
23:12:39 - LiteLLM:DEBUG: utils.py:331 - RAW RESPONSE:
CreateEmbeddingResponse(data=[Embedding(embedding=[-0.011497343890368938, 0.011531440541148186, -0.0040643042884767056, 0.0040199789218604565, -0.010788137093186378, 0.005145163740962744, -0.017893850803375244, -0.010604015551507473, 0.0032221204601228237, -0.010849510319530964, 0.027004437521100044, -0.00840819999575615, 0.02280374802649021, 0.0015880450373515487, -0.022926494479179382, 0.0045962100848555565, 0.030168594792485237, -0.00657039787620306, 0.011285945773124695, -0.018930383026599884, -0.022408228367567062, 0.01642088033258915, 0.0065533495508134365, -0.01102681178599596, -0.016966424882411957, 0.020717041566967964, 0.03295087069272995, -0.030959634110331535, -0.008660513907670975, -0.004487100988


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.



NotFoundError: litellm.NotFoundError: NotFoundError: OpenAIException - Resource not found